## Import

In [5]:
# !pip3 install albumentations ;!pip3 install opencv-python ; !pip3 install tqdm

In [1]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

# torch.multiprocessing import
from torch import multiprocessing

from collections import Counter

import warnings
warnings.filterwarnings(action='ignore') 

c:\Users\DoSungjin\anaconda3\envs\gpu-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 경로지정
import os
os.chdir('../DATA')
os.getcwd()

'c:\\Users\\DoSungjin\\Documents\\GitHub\\Dacon_papering_classification\\DATA'

In [3]:
import torch
# Set the device to CPU or GPU depending on availability
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
torch.cuda.is_available()

True

## Hyperparameter Setting

In [4]:
# hyperparameter
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':25,
    'LEARNING_RATE':0.001,
    'BATCH_SIZE': 64,
    'SEED':42
}

## Fixed RandomSeed

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [6]:
os.getcwd()

'c:\\Users\\DoSungjin\\Documents\\GitHub\\Dacon_papering_classification\\DATA'

In [7]:
import os

# 데이터셋 디렉토리 경로
dataset_dir = "new_train"

# 모든 이미지 파일 경로 리스트
all_img_list = []
folder_list = []
train_file_list = os.listdir(dataset_dir)
for item in train_file_list:
    item_path = os.path.join(dataset_dir, item)
    for file in os.listdir(item_path):
        all_img_list.append(os.path.join(item_path, file))
        folder_list.append(item)

In [8]:
# # folder name list
ori_names = os.listdir('ori_train')

new_names = ['furniture_repair', 'cleaning_mop_holder_repair', 'mold', 'twist', 'rust_contamination', 'wobbling', 'fabric_defect', 'molding_repair', 'stain', 'plaster_repair', 'pollution', 'typographical_error', 'crying', 'defective_joint', 'windowdoor_frame_repair', 'cracking', 'excessive_gap', 'piece', 'damage']

# print(len(ori_names), len(new_names))
# os.chdir('new_train')
# # # folder rename
# for ori_name, new_name in zip(ori_names, new_names):
#     os.rename(ori_name, new_name)

# print(os.listdir())
# os.chdir('../')

In [9]:
for a,b in zip(ori_names, new_names):
    len_a = len(os.listdir(f'ori_train/{a}'))
    len_b = len(os.listdir(f'new_train/{b}'))
    if len_a == len_b:
        print('go on')
    else: 
        print('stop')

go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on


In [10]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('\\')[1])

In [11]:
df

,img_path,label
0,new_train\cleaning_mop_holder_repair\0.png,cleaning_mop_holder_repair
1,new_train\cleaning_mop_holder_repair\1.png,cleaning_mop_holder_repair
2,new_train\cleaning_mop_holder_repair\10.png,cleaning_mop_holder_repair
3,new_train\cleaning_mop_holder_repair\100.png,cleaning_mop_holder_repair
4,new_train\cleaning_mop_holder_repair\101.png,cleaning_mop_holder_repair
...,...,...
3452,new_train\wobbling\53.png,wobbling
3453,new_train\wobbling\6.png,wobbling
3454,new_train\wobbling\7.png,wobbling
3455,new_train\wobbling\8.png,wobbling


In [12]:
# dict value >100
my_dict = Counter(df['label'])
keys_over_100 = []
keys_under_100 = []   
for key, value in my_dict.items():
    # Value 값이 100 이상이면, 해당 Key를 리스트에 추가
    if value >= 100:
        keys_over_100.append(key)
    else:
        keys_under_100.append(key)

# 결과 출력
print(keys_over_100, keys_under_100)

# 새로운 컬럼 생성 및 Over/Under 값 부여
df['c_label'] = df['label'].apply(lambda x: 'Over' if x in keys_over_100 else 'Under')



['cleaning_mop_holder_repair', 'cracking', 'damage', 'mold', 'molding_repair', 'pollution', 'twist', 'typographical_error'] ['crying', 'defective_joint', 'excessive_gap', 'fabric_defect', 'furniture_repair', 'piece', 'plaster_repair', 'rust_contamination', 'stain', 'windowdoor_frame_repair', 'wobbling']


In [1]:
import torch
import gc

# If you are using CUDA
torch.cuda.empty_cache()
gc.collect()

c:\Users\DoSungjin\anaconda3\envs\gpu-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0

In [13]:
df

,img_path,label,c_label
0,new_train\cleaning_mop_holder_repair\0.png,cleaning_mop_holder_repair,Over
1,new_train\cleaning_mop_holder_repair\1.png,cleaning_mop_holder_repair,Over
2,new_train\cleaning_mop_holder_repair\10.png,cleaning_mop_holder_repair,Over
3,new_train\cleaning_mop_holder_repair\100.png,cleaning_mop_holder_repair,Over
4,new_train\cleaning_mop_holder_repair\101.png,cleaning_mop_holder_repair,Over
...,...,...,...
3452,new_train\wobbling\53.png,wobbling,Under
3453,new_train\wobbling\6.png,wobbling,Under
3454,new_train\wobbling\7.png,wobbling,Under
3455,new_train\wobbling\8.png,wobbling,Under


In [1]:
import torch
import gc

# If you are using CUDA
torch.cuda.empty_cache()
gc.collect()

In [14]:
from sklearn.model_selection import train_test_split

train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, stratify=df['label'], random_state=CFG['SEED'])


## Label-Encoding

In [15]:
le = preprocessing.LabelEncoder()
train['c_label'] = le.fit_transform(train['c_label'])
val['c_label'] = le.transform(val['c_label'])
train

,img_path,label,c_label
1182,new_train\damage\356.png,damage,0
2455,new_train\pollution\137.png,pollution,0
3226,new_train\twist\92.png,twist,0
2742,new_train\pollution\396.png,pollution,0
2134,new_train\mold\63.png,mold,0
...,...,...,...
112,new_train\cleaning_mop_holder_repair\2.png,cleaning_mop_holder_repair,0
315,new_train\cracking\105.png,cracking,0
1932,new_train\fabric_defect\21.png,fabric_defect,1
1089,new_train\damage\272.png,damage,0


## CustomDataset

In [16]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [2]:
import torch
import gc

# If you are using CUDA
torch.cuda.empty_cache()
gc.collect()

0

In [17]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

train_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.HorizontalFlip(p=0.5),  # Flip the image horizontally with probability 0.5
    A.Rotate(limit=10, p=0.5),  # Rotate the image by -10 to 10 degrees with probability 0.5
    A.RandomBrightnessContrast(p=0.2),  # Add random brightness and contrast with probability 0.2
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])


In [18]:
train

,img_path,label,c_label
1182,new_train\damage\356.png,damage,0
2455,new_train\pollution\137.png,pollution,0
3226,new_train\twist\92.png,twist,0
2742,new_train\pollution\396.png,pollution,0
2134,new_train\mold\63.png,mold,0
...,...,...,...
112,new_train\cleaning_mop_holder_repair\2.png,cleaning_mop_holder_repair,0
315,new_train\cracking\105.png,cracking,0
1932,new_train\fabric_defect\21.png,fabric_defect,1
1089,new_train\damage\272.png,damage,0


In [19]:
train['img_path'].values

array(['new_train\\damage\\356.png', 'new_train\\pollution\\137.png',
       'new_train\\twist\\92.png', ...,
       'new_train\\fabric_defect\\21.png', 'new_train\\damage\\272.png',
       'new_train\\crying\\5.png'], dtype=object)

In [20]:
train_dataset = CustomDataset(train['img_path'].values, train['c_label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)


val_dataset = CustomDataset(val['img_path'].values, val['c_label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Model Define

In [23]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

# Train

# Train_2

In [36]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    # 임계값 후보 리스트
    threshold_list = np.linspace(0, 1, 101)
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.long).to(device)
        
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, threshold_list, device)  # 임계값 최적화 함수 호출
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model


def validation(model, criterion, val_loader, threshold_list, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []
    best_threshold = None  # 최적의 임계값
    best_f1_score = 0  # 최적의 F1-score
    
    with torch.no_grad():
        for threshold in threshold_list:
            val_loss = []
            preds, true_labels = [], []
            
            for imgs, labels in tqdm(iter(val_loader)):
                imgs = imgs.float().to(device)
                labels = labels.type(torch.long).to(device)
                
                pred = model(imgs)
                loss = criterion(pred, labels)
                
                # 임계값을 적용하여 예측 결과 계산
                pred_thresholded = (pred.argmax(1) >= threshold).to(torch.long)
                
                preds += pred_thresholded.detach().cpu().numpy().tolist()
                true_labels += labels.detach().cpu().numpy().tolist()
                
                val_loss.append(loss.item())
            
            val_loss_avg = np.mean(val_loss)
            f1 = f1_score(true_labels, preds, average='weighted')
            
            if f1 > best_f1_score:
                best_f1_score = f1
                best_threshold = threshold
            
    print(f"Best threshold: {best_threshold}, Best F1-score: {best_f1_score}")
    
    return val_loss_avg, best_f1_score


## Run!!

In [26]:
model = BaseModel()
model.eval()
optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

100%|██████████| 11/11 [00:09<00:00,  1.11it/s]


Epoch [1], Train Loss : [0.36281] Val Loss : [0.30213] Val Weighted F1 Score : [0.89864]


100%|██████████| 11/11 [00:05<00:00,  2.12it/s]


Epoch [2], Train Loss : [0.18949] Val Loss : [0.34551] Val Weighted F1 Score : [0.91967]


100%|██████████| 11/11 [00:05<00:00,  1.98it/s]


...

Epoch [23], Train Loss : [0.00229] Val Loss : [0.66649] Val Weighted F1 Score : [0.91193]


100%|██████████| 11/11 [00:05<00:00,  2.12it/s]


Epoch [24], Train Loss : [0.00204] Val Loss : [0.66071] Val Weighted F1 Score : [0.91072]


100%|██████████| 11/11 [00:05<00:00,  2.11it/s]

Epoch [25], Train Loss : [0.00410] Val Loss : [0.66868] Val Weighted F1 Score : [0.91193]
Epoch 00025: reducing learning rate of group 0 to 7.8125e-06.


## Inference

In [27]:
test = pd.read_csv('test.csv')
test

,id,img_path
0,TEST_000,./test/000.png
1,TEST_001,./test/001.png
2,TEST_002,./test/002.png
3,TEST_003,./test/003.png
4,TEST_004,./test/004.png
...,...,...
787,TEST_787,./test/787.png
788,TEST_788,./test/788.png
789,TEST_789,./test/789.png
790,TEST_790,./test/790.png


In [28]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [29]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [30]:

preds = inference(infer_model, test_loader, device)

100%|██████████| 13/13 [00:17<00:00,  1.36s/it]


In [34]:
preds

array(['Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over',
       'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over',
       'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over',
       'Over', 'Over', 'Under', 'Over', 'Over', 'Over', 'Over', 'Over',
       'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over',
       'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over',
       'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over',
       'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over',
       'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over',
       'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over',
       'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over',
       'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over',
       'Over', 'Over', 'Over', 'Over', 'Under', 'Over', 'Over', 'Over',
       'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over', 'Over',
    

In [35]:
test_2 = pd.read_csv('sample_submission.csv')
test_2['c_label'] = preds
test_2

,id,label,c_label
0,TEST_000,훼손,Over
1,TEST_001,훼손,Over
2,TEST_002,훼손,Over
3,TEST_003,훼손,Over
4,TEST_004,훼손,Over
...,...,...,...
787,TEST_787,훼손,Over
788,TEST_788,훼손,Over
789,TEST_789,훼손,Over
790,TEST_790,훼손,Under


## Submission

In [ ]:
submit = pd.read_csv('sample_submission.csv')
submit['label'] = preds

In [ ]:
submit

,id,label
0,TEST_000,damage
1,TEST_001,pollution
2,TEST_002,damage
3,TEST_003,molding_repair
4,TEST_004,pollution
...,...,...
787,TEST_787,damage
788,TEST_788,damage
789,TEST_789,pollution
790,TEST_790,pollution


In [ ]:
# result folder 생성
from datetime import datetime, timezone, timedelta

kst = timezone(timedelta(hours=9))
train_serial =  datetime.now(kst).strftime('%Y%m%d_%H%M%S')

Record_path = os.path.join('result', train_serial)

os.makedirs(Record_path, exist_ok=True)




In [ ]:
# Dictionary to map old column names to new column names
name_map = dict(zip(new_names, ori_names))
name_map


# Rename columns using the rename() method
submit_test = submit.replace(name_map)
submit_test

,id,label
0,TEST_000,훼손
1,TEST_001,오염
2,TEST_002,훼손
3,TEST_003,몰딩수정
4,TEST_004,오염
...,...,...
787,TEST_787,훼손
788,TEST_788,훼손
789,TEST_789,오염
790,TEST_790,오염


In [ ]:
submit_test.to_csv(os.path.join(Record_path,'submission.csv'), index=False)

In [ ]:
submit_test

,id,label
0,TEST_000,훼손
1,TEST_001,오염
2,TEST_002,훼손
3,TEST_003,몰딩수정
4,TEST_004,오염
...,...,...
787,TEST_787,훼손
788,TEST_788,훼손
789,TEST_789,오염
790,TEST_790,오염


In [ ]:
# model information 저장
import json
model_info = {'CFG' : CFG, 'model' : str(model), 'optimizer' : str(optimizer), 'scheduler' : str(scheduler), 'train_serial' : train_serial}
with open(os.path.join(Record_path,'model_info.json'), 'w') as f:
    json.dump(model_info, f)